## 4.2 Extract hyperlinks of song pages

In [61]:
import pandas as pd
import re
import pprint
import requests
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
html = requests.get('https://www.lyrics.com/artist/Cher').text

In [3]:
# Get the hyperlink to spice
hyper_list = re.findall(pattern='<a href="(/lyric/[^"]+)', string=html)

In [4]:
prefix_url = "https://www.lyrics.com"

complete_url_list = []
song_name_list = []
artist_list = []

for hyper_link in hyper_list:
    complete_url = prefix_url + hyper_link
    complete_url_list.append(complete_url)
    song_name = re.findall(pattern='Cher/([^"]+)', string=hyper_link)
    song_name_list.append(song_name[0])
    artist = 'Cher'
    artist_list.append(artist)

In [5]:
dic = {'song_name':song_name_list , 'artist_y': artist_list , 'song_url':complete_url_list}

In [6]:
df = pd.DataFrame(dic)

In [7]:
df.drop_duplicates(['song_name'], inplace=True)

In [8]:
df.reset_index(drop=True, inplace=True)

In [42]:
df

,song_name,artist_y,song_url
0,Believe,Cher,https://www.lyrics.com/lyric/36071465/Cher/Bel...
1,Strong+Enough,Cher,https://www.lyrics.com/lyric/36054417/Cher/Str...
2,Dov%27%C3%A8+l%27amore,Cher,https://www.lyrics.com/lyric/36037592/Cher/Dov...
3,Dead+Ringer+for+Love,Cher,https://www.lyrics.com/lyric/34909213/Cher/Dea...
4,If+I+Could+Turn+Back+Time,Cher,https://www.lyrics.com/lyric/34960194/Cher/If+...
...,...,...,...
538,Chiquitita+%5BSpanish+Version%5D,Cher,https://www.lyrics.com/lyric/37103698/Cher/Chi...
539,Walls,Cher,https://www.lyrics.com/lyric/38437151/Cher/Walls
540,Stop+Crying+Your+Heart+Out,Cher,https://www.lyrics.com/lyric/37763506/Cher/Sto...
541,The+Look,Cher,https://www.lyrics.com/lyric/5352359/Cher/The+...


In [9]:
df.loc[2]

song_name                               Dov%27%C3%A8+l%27amore
artist_y                                                  Cher
song_url     https://www.lyrics.com/lyric/36037592/Cher/Dov...
Name: 2, dtype: object

## Download the song pages as text files

In [10]:
save_path = '/home/damoon/damoon_spiced_academy/04_week/song_lyrics/Cher'
for i in range(0, df.shape[0]): 
    file_name = df['song_name'][i]
    completeName = os.path.join(save_path, file_name)
    song_html = requests.get(df['song_url'][i]).text
    with open (completeName, "w") as f:
        f.write(song_html)
    f.close()


KeyboardInterrupt: 

## 4.3 Download and extract the song lyrics and clean the lyrics

In [11]:
from bs4 import BeautifulSoup

In [ ]:

def clean_lyrics(string):
    str_no_tag = re.sub('<.*?>' ,'', string)
    str_no_nextline =re.sub('\n', ' ', str_no_tag)
    str_no_nonalpha =re.sub('\W', ' ', str_no_nextline)
    return(str_no_nonalpha)

In [22]:
corpus = []
for fn in os.listdir('song_lyrics/Cher/'):
    text = open('song_lyrics/Cher/' + fn).read()
    text_soup = BeautifulSoup(text, 'html.parser')
    lyrics = text_soup.pre.get_text()
    lyrics_cleaned = clean_lyrics(lyrics)
    corpus.append(lyrics_cleaned) 
    
    #lyrics_list.append(text_soup.body.find_all(class_ = 'lyric-body')[0].get_text()) 

In [27]:
labels = ["Cher"] * len(corpus)

## Vectorization

In [46]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names(), index=labels)

In [47]:
X_df

,about,above,afraid,after,ain,all,alone,am,amo,amore,...,world,worries,worth,would,write,yeah,yes,you,your,yourself
Cher,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,9,0,0
Cher,0,0,0,0,0,0,1,0,1,4,...,0,0,0,0,0,0,0,13,6,0
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,10,6,0
Cher,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,11,1,0
Cher,0,0,0,0,0,2,1,3,0,0,...,2,0,0,1,0,0,0,3,0,0
Cher,0,0,0,0,0,1,0,0,0,0,...,0,1,1,0,0,2,0,44,17,2
Cher,1,0,0,0,1,0,0,0,0,0,...,0,0,3,0,1,0,0,23,5,0
Cher,0,0,0,19,0,1,0,1,0,0,...,0,0,0,0,0,0,0,34,0,0
Cher,0,2,0,0,0,2,1,2,0,0,...,0,0,0,0,0,0,0,12,30,0


In [48]:
X_df.shape

(9, 386)

In [49]:
X

<9x386 sparse matrix of type '<class 'numpy.int64'>'
	with 697 stored elements in Compressed Sparse Row format>

In [50]:
type(X)

scipy.sparse.csr.csr_matrix

In [52]:
## Using Stopwords

vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(corpus)
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names(), index=labels)

In [53]:
X_df

,afraid,ain,amo,amore,anymore,arms,aside,ask,asleep,away,...,weep,whispered,woman,won,world,worries,worth,write,yeah,yes
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cher,0,0,1,4,0,1,0,0,0,4,...,0,1,0,0,0,0,0,0,0,0
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,3
Cher,1,0,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,0
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2,0,0,0,0,0
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,2,0
Cher,0,1,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,3,1,0,0
Cher,0,0,0,0,12,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cher,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [55]:
# Remove words that are in 80% of documents
vectorizer = CountVectorizer(max_df=0.8)
X = vectorizer.fit_transform(corpus)
X_df = pd.DataFrame(X.todense(), columns=vectorizer.get_feature_names(), index=labels)

In [56]:
vectorizer.stop_words_

{'and', 'me', 'on', 'that', 'the', 'to', 'you'}

In [59]:
from sklearn.feature_extraction.text import TfidfTransformer

tf = TfidfTransformer() 
#Use the CountVectorised data
X_norm = tf.fit_transform(X)

In [60]:
X_norm_df=pd.DataFrame(X_norm.todense(), columns=vectorizer.get_feature_names(), index=labels)
X_norm_df

,about,above,afraid,after,ain,all,alone,am,amo,amore,...,won,world,worries,worth,would,write,yeah,yes,your,yourself
Cher,0.083452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029876,0.000000,0.040682,0.162729,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.126907,0.00000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.113313,0.117826,0.00000
Cher,0.000000,0.000000,0.074393,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.038678,0.00000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.077622,0.043926,0.131778,0.000000,0.000000,...,0.000000,0.119629,0.00000,0.000000,0.059814,0.000000,0.00000,0.000000,0.000000,0.00000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.016909,0.000000,0.000000,0.000000,0.000000,...,0.022011,0.000000,0.02606,0.022011,0.000000,0.000000,0.05212,0.000000,0.230333,0.05212
Cher,0.030970,0.000000,0.000000,0.000000,0.036667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.092909,0.000000,0.036667,0.00000,0.000000,0.095318,0.00000
Cher,0.000000,0.000000,0.000000,0.512898,0.000000,0.017516,0.000000,0.019824,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000
Cher,0.000000,0.065016,0.000000,0.000000,0.000000,0.042186,0.023873,0.047746,0.000000,0.000000,...,0.027457,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.507041,0.00000


In [63]:
vectorizer = TfidfVectorizer()

In [64]:
X_vec = vectorizer.fit_transform(corpus)
X_vec_df = pd.DataFrame(X_vec.todense(), columns=vectorizer.get_feature_names(), index=labels)

In [65]:
X_vec_df

,about,above,afraid,after,ain,all,alone,am,amo,amore,...,world,worries,worth,would,write,yeah,yes,you,your,yourself
Cher,0.081007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.165397,0.000000,0.000000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028709,0.000000,0.039094,0.156374,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.194761,0.121951,0.000000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111518,0.142455,0.115959,0.000000
Cher,0.000000,0.000000,0.065576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.276435,0.034094,0.000000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.075792,0.042891,0.128672,0.000000,0.000000,...,0.116809,0.000000,0.000000,0.058405,0.000000,0.000000,0.000000,0.067146,0.000000,0.000000
Cher,0.000000,0.000000,0.000000,0.000000,0.000000,0.015085,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.023248,0.019636,0.000000,0.000000,0.046496,0.000000,0.392008,0.205479,0.046496
Cher,0.028501,0.000000,0.000000,0.000000,0.033744,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.085503,0.000000,0.033744,0.000000,0.000000,0.297427,0.087720,0.000000
Cher,0.000000,0.000000,0.000000,0.480381,0.000000,0.016405,0.000000,0.018567,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.329431,0.000000,0.000000
Cher,0.000000,0.063938,0.000000,0.000000,0.000000,0.041486,0.023477,0.046954,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.147015,0.498630,0.000000


In [ ]:
james = 